# 🧠 PYUSD Colab Starter Notebook
### Query Ethereum Blockchain (BigQuery) and Export to Google Sheets

In [ ]:
# ⚙️ STEP 1: Install Required Libraries
!pip install --quiet --upgrade gspread gspread_dataframe google-auth google-cloud-bigquery pandas

In [ ]:
# 📁 STEP 2: Import Libraries
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
from google.cloud import bigquery
from google.auth import default

# 🔐 STEP 3: Authenticate
auth.authenticate_user()
creds, _ = default()

# Initialize BigQuery and Google Sheets clients
bq_client = bigquery.Client(credentials=creds, project=creds.project_id)
gc = gspread.authorize(creds)

print("✅ Authenticated with Google Cloud")

In [ ]:
# 📊 STEP 4: Querying BigQuery Ethereum Data
query = """
SELECT 
  block_timestamp,
  transaction_hash,
  from_address,
  to_address,
  value / 1e18 as pyusd_value
FROM 
  `bigquery-public-data.crypto_ethereum.token_transfers`
WHERE 
  token_address = '0x6c3ea9036406852006290770bedfcaba0e23a0e8'
  AND block_timestamp > TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY)
ORDER BY block_timestamp DESC
LIMIT 1000
"""

df = bq_client.query(query).to_dataframe()
print(f"✅ Retrieved {len(df)} PYUSD transfers")
df.head()

In [ ]:
# 📤 STEP 5: Exporting to Google Sheets
sheet = gc.create("PYUSD_Transfers_Export")
worksheet = sheet.get_worksheet(0)

# Export dataframe to the sheet
set_with_dataframe(worksheet, df)
print("✅ Exported to Google Sheets:", sheet.url)